---
## 1. Teoretický Úvod

### 1.1 Proč Klasifikace místo Regrese?

| Aspekt | Regrese | Klasifikace |
|--------|---------|-------------|
| **Výstup** | Přesná cena ($150.23) | Směr (UP/DOWN) |
| **Praktičnost** | Obtížně použitelné | Přímý obchodní signál |
| **Citlivost na outliery** | Vysoká (NVDA +200%) | Nízká |
| **Evaluace** | MAE, RMSE | Accuracy, F1 |

### 1.2 Proč Ternární Klasifikace?

**Binární** (UP/DOWN):
- ❌ Ignoruje transakční náklady
- ❌ Malé pohyby (±1%) vedou k falešným signálům

**Ternární** (DOWN/HOLD/UP):
- ✅ HOLD třída zachycuje neutrální zónu
- ✅ Práh ±3% pokrývá transakční náklady
- ✅ Realističtější obchodní signály

### 1.3 Práh 3% - Zdůvodnění

$$\text{Threshold} = \text{Bid-Ask Spread} + \text{Broker Fee} + \text{Slippage}$$

Typicky:
- Bid-Ask: ~0.5%
- Broker: ~0.5%
- Slippage: ~1%
- **Rezerva**: ~1%

**Celkem: ~3%**

### 1.4 Formální Definice

Pro výnos $r_t = \frac{P_{t+1} - P_t}{P_t}$ definujeme:

$$y_t = \begin{cases}
0 \text{ (DOWN)} & \text{pokud } r_t < -0.03 \\
1 \text{ (HOLD)} & \text{pokud } -0.03 \leq r_t \leq 0.03 \\
2 \text{ (UP)} & \text{pokud } r_t > 0.03
\end{cases}$$

### 1.5 Random Forest pro Klasifikaci

$$P(y = k | x) = \frac{1}{B}\sum_{b=1}^{B} \mathbb{1}[T_b(x) = k]$$

kde:
- $B$ = počet stromů
- $T_b(x)$ = predikce b-tého stromu
- $\mathbb{1}[\cdot]$ = indikátorová funkce

---
## 2. Setup Prostředí

In [ ]:
# Instalace (pro Colab)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.notebook import tqdm
import warnings
import os
import joblib
import json

# Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)

warnings.filterwarnings('ignore')
np.random.seed(42)

print("✓ Knihovny načteny")

In [ ]:
# Připojení Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    print("ℹ️ Lokální prostředí")

# Cesty
DATA_PATH = f"{DRIVE_PATH}/data"
MODEL_PATH = f"{DRIVE_PATH}/models"
PREDICTIONS_PATH = f"{DATA_PATH}/predictions"
os.makedirs(MODEL_PATH, exist_ok=True)
os.makedirs(PREDICTIONS_PATH, exist_ok=True)

---
## 3. Konfigurace

In [ ]:
# ============================================================
# KONFIGURACE KLASIFIKÁTORU
# ============================================================

# Práh pro ternární klasifikaci
THRESHOLD = 0.03  # 3%

# Názvy tříd
CLASS_NAMES = ['DOWN', 'HOLD', 'UP']
CLASS_MAPPING = {0: 'DOWN', 1: 'HOLD', 2: 'UP'}

# Random Forest hyperparametry
RF_PARAMS = {
    'n_estimators': 200,
    'max_depth': 12,
    'min_samples_split': 10,
    'min_samples_leaf': 5,
    'class_weight': 'balanced',  # Vyvážení nebalancovaných tříd
    'random_state': 42,
    'n_jobs': -1
}

# Features pro klasifikaci
FEATURE_COLS = [
    # OHLCV
    'open', 'high', 'low', 'close', 'volume',
    # Technické
    'returns', 'volatility_12m', 'rsi_14',
    'macd', 'macd_signal', 'macd_hist',
    'sma_3', 'sma_6', 'sma_12',
    'ema_3', 'ema_6', 'ema_12',
    'volume_change', 'price_momentum',
    # Fundamenty
    'PE', 'PB', 'PS', 'EV_EBITDA',
    'ROE', 'ROA', 'Profit_Margin',
    'Debt_to_Equity', 'Current_Ratio',
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'
]

print("📊 KONFIGURACE KLASIFIKÁTORU")
print("="*50)
print(f"🎯 Práh: ±{THRESHOLD*100:.0f}%")
print(f"📊 Třídy: {CLASS_NAMES}")
print(f"🌲 RF estimators: {RF_PARAMS['n_estimators']}")
print(f"📈 Features: {len(FEATURE_COLS)}")

---
## 4. Načtení a Příprava Dat

In [ ]:
# Načtení kompletního datasetu z Notebooku 03
complete_path = f"{DATA_PATH}/complete/all_sectors_complete_10y.csv"
df = pd.read_csv(complete_path, parse_dates=['date'])

print(f"📈 Kompletní Dataset:")
print(f"   Záznamů: {len(df):,}")
print(f"   Tickerů: {df['ticker'].nunique()}")
print(f"   Sektorů: {df['sector'].nunique()}")
print(f"   Období: {df['date'].min().strftime('%Y-%m')} → {df['date'].max().strftime('%Y-%m')}")

In [ ]:
def create_target_variable(df: pd.DataFrame, threshold: float = 0.03) -> pd.DataFrame:
    """
    Vytvoří ternární target proměnnou na základě budoucího výnosu.
    
    Target je posun o 1 měsíc dopředu (predikujeme příští měsíc).
    
    Args:
        df: DataFrame s OHLCV daty
        threshold: Práh pro klasifikaci (default 3%)
    
    Returns:
        DataFrame s přidaným target sloupcem
    """
    result = df.copy()
    
    # Pro každý ticker vytvoříme future_return
    result['future_return'] = np.nan
    result['target'] = np.nan
    
    for ticker in tqdm(result['ticker'].unique(), desc="Vytváření target"):
        mask = result['ticker'] == ticker
        ticker_data = result.loc[mask].sort_values('date').copy()
        
        # Future return = (close[t+1] - close[t]) / close[t]
        ticker_data['future_return'] = ticker_data['close'].shift(-1) / ticker_data['close'] - 1
        
        # Ternární klasifikace
        conditions = [
            ticker_data['future_return'] < -threshold,  # DOWN
            ticker_data['future_return'] > threshold,   # UP
        ]
        choices = [0, 2]  # DOWN=0, UP=2
        ticker_data['target'] = np.select(conditions, choices, default=1)  # HOLD=1
        
        result.loc[mask, 'future_return'] = ticker_data['future_return'].values
        result.loc[mask, 'target'] = ticker_data['target'].values
    
    # Odstranění posledního měsíce (nemáme future return)
    result = result.dropna(subset=['target'])
    result['target'] = result['target'].astype(int)
    
    return result

# Vytvoření target
print("🎯 Vytváření target proměnné...")
df = create_target_variable(df, THRESHOLD)

print(f"\n✓ Záznamů po vytvoření target: {len(df):,}")

In [ ]:
# Distribuce tříd
class_dist = df['target'].value_counts().sort_index()

print("\n📊 DISTRIBUCE TŘÍD")
print("="*40)
for cls, count in class_dist.items():
    pct = count / len(df) * 100
    bar = '█' * int(pct / 2)
    print(f"   {CLASS_MAPPING[cls]}: {count:>6} ({pct:>5.1f}%) {bar}")

In [ ]:
# Vizualizace distribuce tříd
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1. Pie chart
ax1 = axes[0]
colors = ['#e74c3c', '#f39c12', '#2ecc71']
ax1.pie(class_dist, labels=CLASS_NAMES, colors=colors, autopct='%1.1f%%',
        startangle=90, explode=[0.02, 0.02, 0.02])
ax1.set_title('Distribuce Tříd', fontweight='bold')

# 2. Distribuce future_return
ax2 = axes[1]
ax2.hist(df['future_return'].dropna(), bins=50, color='steelblue', 
         edgecolor='black', alpha=0.7)
ax2.axvline(-THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'-{THRESHOLD*100:.0f}%')
ax2.axvline(THRESHOLD, color='green', linestyle='--', linewidth=2, label=f'+{THRESHOLD*100:.0f}%')
ax2.axvline(0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Měsíční Výnos')
ax2.set_ylabel('Frekvence')
ax2.set_title('Distribuce Měsíčních Výnosů', fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/class_distribution.png", dpi=150, bbox_inches='tight')
plt.show()

---
## 5. Příprava Trénovacích Dat

In [ ]:
def prepare_classification_data(df: pd.DataFrame, feature_cols: list):
    """
    Připraví data pro klasifikaci.
    
    Returns:
        X, y, valid_features, clean_df
    """
    # Dostupné features
    available_features = [f for f in feature_cols if f in df.columns]
    missing_features = set(feature_cols) - set(available_features)
    
    if missing_features:
        print(f"⚠️ Chybějící features: {missing_features}")
    
    # Odstranění NaN
    clean_df = df.dropna(subset=available_features + ['target'])
    
    print(f"📊 Příprava dat:")
    print(f"   Dostupné features: {len(available_features)}")
    print(f"   Záznamů po čištění: {len(clean_df):,}")
    
    X = clean_df[available_features].values
    y = clean_df['target'].values
    
    return X, y, available_features, clean_df

# Příprava
X, y, valid_features, clean_df = prepare_classification_data(df, FEATURE_COLS)

print(f"\n✓ X shape: {X.shape}")
print(f"✓ y shape: {y.shape}")

---
## 6. Chronologický Train/Test Split

### 6.1 Proč Chronologický Split?

**Náhodný split je CHYBNÝ** pro časové řady:
- Data leakage: Model vidí budoucnost
- Nerealistické výsledky: Vyšší accuracy než v praxi

**Chronologický split:**
- Train: 2015-2022 (80%)
- Test: 2023-2025 (20%)

In [ ]:
# Chronologický split
clean_df_sorted = clean_df.sort_values('date').reset_index(drop=True)

# 80/20 split
split_idx = int(len(clean_df_sorted) * 0.8)
split_date = clean_df_sorted.iloc[split_idx]['date']

train_df = clean_df_sorted.iloc[:split_idx]
test_df = clean_df_sorted.iloc[split_idx:]

X_train = train_df[valid_features].values
y_train = train_df['target'].values
X_test = test_df[valid_features].values
y_test = test_df['target'].values

print(f"📊 Chronologický Split:")
print(f"   Split datum: {split_date.strftime('%Y-%m')}")
print(f"   Train: {len(X_train):,} samples ({train_df['date'].min().strftime('%Y-%m')} → {train_df['date'].max().strftime('%Y-%m')})")
print(f"   Test: {len(X_test):,} samples ({test_df['date'].min().strftime('%Y-%m')} → {test_df['date'].max().strftime('%Y-%m')})")

In [ ]:
# Standardizace
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardizovány")

---
## 7. Trénování Klasifikátoru

In [ ]:
%%time

# Trénování Random Forest Classifier
print("🚀 Trénování Random Forest Classifier...")
print(f"   Třídy: {CLASS_NAMES}")
print(f"   Features: {len(valid_features)}")
print(f"   Training samples: {len(X_train):,}")
print()

clf = RandomForestClassifier(**RF_PARAMS)
clf.fit(X_train_scaled, y_train)

print("\n✅ Model natrénován!")

---
## 8. Evaluace Modelu

### 8.1 Metriky

| Metrika | Formule | Interpretace |
|---------|---------|-------------|
| **Accuracy** | $\frac{TP + TN}{Total}$ | Celková přesnost |
| **Precision** | $\frac{TP}{TP + FP}$ | Přesnost pozitivních predikcí |
| **Recall** | $\frac{TP}{TP + FN}$ | Pokrytí skutečných pozitivních |
| **F1-Score** | $2 \cdot \frac{P \cdot R}{P + R}$ | Harmonický průměr P a R |

In [ ]:
# Predikce
y_pred = clf.predict(X_test_scaled)
y_pred_proba = clf.predict_proba(X_test_scaled)

# Metriky
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("📊 EVALUACE MODELU")
print("="*50)
print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

In [ ]:
# Classification Report
print("\n📊 CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Absolutní hodnoty
ax1 = axes[0]
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)
disp1.plot(ax=ax1, cmap='Blues', values_format='d')
ax1.set_title('Confusion Matrix (Absolutní)', fontweight='bold')

# 2. Normalizované
ax2 = axes[1]
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=CLASS_NAMES)
disp2.plot(ax=ax2, cmap='Blues', values_format='.2f')
ax2.set_title('Confusion Matrix (Normalizovaná)', fontweight='bold')

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/confusion_matrix.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {DATA_PATH}/confusion_matrix.png")

---
## 9. Feature Importance

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'Feature': valid_features,
    'Importance': clf.feature_importances_
}).sort_values('Importance', ascending=False)

print("📊 FEATURE IMPORTANCE (Top 15)")
print("="*50)
for i, (_, row) in enumerate(importance_df.head(15).iterrows()):
    bar = '█' * int(row['Importance'] * 100)
    print(f"{i+1:>2}. {row['Feature']:<25} {row['Importance']:.4f} {bar}")

In [ ]:
# Vizualizace feature importance
fig, ax = plt.subplots(figsize=(10, 8))

top_n = 15
top_features = importance_df.head(top_n)

colors = plt.cm.viridis(np.linspace(0, 0.8, top_n))
bars = ax.barh(top_features['Feature'], top_features['Importance'], color=colors)

ax.set_xlabel('Importance')
ax.set_title('Top 15 Feature Importance pro Klasifikaci', fontsize=14, fontweight='bold')
ax.invert_yaxis()

for bar, val in zip(bars, top_features['Importance']):
    ax.text(val + 0.002, bar.get_y() + bar.get_height()/2, 
            f'{val:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/classifier_feature_importance.png", dpi=150, bbox_inches='tight')
plt.show()

---
## 10. Analýza Trading Strategie

### 10.1 Backtesting

Simulujeme jednoduchou strategii:
- **UP predikce**: Kup akcie
- **DOWN predikce**: Prodej (short)
- **HOLD predikce**: Drž

In [ ]:
def analyze_trading_strategy(y_true, y_pred, returns):
    """
    Analyzuje trading strategii založenou na klasifikátoru.
    
    Strategie:
    - UP (2): Investuj (gain = return)
    - DOWN (0): Short (gain = -return)
    - HOLD (1): Neinvestuj (gain = 0)
    """
    # Výpočet zisku pro každou predikci
    gains = np.zeros_like(returns)
    
    # UP predikce - long
    gains[y_pred == 2] = returns[y_pred == 2]
    
    # DOWN predikce - short
    gains[y_pred == 0] = -returns[y_pred == 0]
    
    # HOLD - nic
    gains[y_pred == 1] = 0
    
    # Statistiky
    total_trades = np.sum(y_pred != 1)
    profitable_trades = np.sum(gains > 0)
    losing_trades = np.sum(gains < 0)
    
    win_rate = profitable_trades / total_trades if total_trades > 0 else 0
    avg_gain = np.mean(gains[y_pred != 1]) if total_trades > 0 else 0
    total_return = np.sum(gains)
    
    # Buy & Hold pro srovnání
    buy_hold_return = np.sum(returns)
    
    return {
        'total_trades': total_trades,
        'profitable_trades': profitable_trades,
        'losing_trades': losing_trades,
        'win_rate': win_rate,
        'avg_gain_per_trade': avg_gain,
        'strategy_return': total_return,
        'buy_hold_return': buy_hold_return,
        'outperformance': total_return - buy_hold_return
    }

# Analýza
test_returns = test_df['future_return'].values
strategy = analyze_trading_strategy(y_test, y_pred, test_returns)

print("📈 TRADING STRATEGIE ANALÝZA")
print("="*50)
print(f"   Celkem obchodů: {strategy['total_trades']:,}")
print(f"   Ziskových: {strategy['profitable_trades']:,}")
print(f"   Ztrátových: {strategy['losing_trades']:,}")
print(f"   Win Rate: {strategy['win_rate']*100:.1f}%")
print(f"   Průměrný zisk/obchod: {strategy['avg_gain_per_trade']*100:.2f}%")
print()
print(f"   📊 Strategie Return: {strategy['strategy_return']*100:.1f}%")
print(f"   📊 Buy & Hold Return: {strategy['buy_hold_return']*100:.1f}%")
print(f"   📊 Outperformance: {strategy['outperformance']*100:.1f}%")

---
## 11. Trénování per Sektor

In [ ]:
# Trénování modelu pro každý sektor
sector_results = {}
sector_models = {}

print("🏭 TRÉNOVÁNÍ PER SEKTOR")
print("="*70)

for sector in clean_df['sector'].unique():
    print(f"\n📊 {sector}")
    print("-"*50)
    
    # Filtrování dat pro sektor
    sector_df = clean_df[clean_df['sector'] == sector].sort_values('date').reset_index(drop=True)
    
    # Split
    split_idx = int(len(sector_df) * 0.8)
    train_sector = sector_df.iloc[:split_idx]
    test_sector = sector_df.iloc[split_idx:]
    
    X_train_s = train_sector[valid_features].values
    y_train_s = train_sector['target'].values
    X_test_s = test_sector[valid_features].values
    y_test_s = test_sector['target'].values
    
    # Standardizace
    scaler_s = StandardScaler()
    X_train_s_scaled = scaler_s.fit_transform(X_train_s)
    X_test_s_scaled = scaler_s.transform(X_test_s)
    
    # Trénování
    clf_s = RandomForestClassifier(**RF_PARAMS)
    clf_s.fit(X_train_s_scaled, y_train_s)
    
    # Evaluace
    y_pred_s = clf_s.predict(X_test_s_scaled)
    
    acc = accuracy_score(y_test_s, y_pred_s)
    f1 = f1_score(y_test_s, y_pred_s, average='weighted')
    
    sector_results[sector] = {
        'accuracy': acc,
        'f1': f1,
        'train_samples': len(X_train_s),
        'test_samples': len(X_test_s)
    }
    
    sector_models[sector] = {
        'model': clf_s,
        'scaler': scaler_s
    }
    
    print(f"   Train: {len(X_train_s):,} | Test: {len(X_test_s):,}")
    print(f"   Accuracy: {acc:.4f} | F1: {f1:.4f}")

# Shrnutí
print("\n" + "="*70)
print("📊 SHRNUTÍ PER SEKTOR")
print("="*70)
print(f"{'Sektor':<15} {'Accuracy':>10} {'F1':>10} {'Train':>10} {'Test':>10}")
print("-"*70)
for sector, res in sector_results.items():
    print(f"{sector:<15} {res['accuracy']:>10.4f} {res['f1']:>10.4f} {res['train_samples']:>10,} {res['test_samples']:>10,}")

---
## 12. Uložení Modelů

In [ ]:
# Uložení globálního modelu
global_model_path = f"{MODEL_PATH}/price_classifier_global.pkl"
global_scaler_path = f"{MODEL_PATH}/classifier_scaler_global.pkl"

joblib.dump(clf, global_model_path)
joblib.dump(scaler, global_scaler_path)

print(f"💾 Globální model: {global_model_path}")
print(f"💾 Globální scaler: {global_scaler_path}")

In [ ]:
# Uložení sektorových modelů
for sector, model_dict in sector_models.items():
    model_path = f"{MODEL_PATH}/{sector}_price_classifier.pkl"
    scaler_path = f"{MODEL_PATH}/{sector}_classifier_scaler.pkl"
    
    joblib.dump(model_dict['model'], model_path)
    joblib.dump(model_dict['scaler'], scaler_path)
    
    print(f"💾 {sector}: {model_path}")

In [ ]:
# Uložení metadat
metadata = {
    'features': valid_features,
    'classes': CLASS_NAMES,
    'threshold': THRESHOLD,
    'rf_params': RF_PARAMS,
    'global_accuracy': accuracy,
    'global_f1': f1,
    'sector_results': sector_results,
    'created': datetime.now().isoformat()
}

metadata_path = f"{MODEL_PATH}/price_classifier_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"\n💾 Metadata: {metadata_path}")

---
## 13. Shrnutí

### ✅ Dokončeno:

| Úkol | Status |
|------|--------|
| Definice ternární klasifikace | ✅ |
| Příprava trénovacích dat | ✅ |
| Trénování globálního modelu | ✅ |
| Trénování sektorových modelů | ✅ |
| Evaluace (Accuracy, F1, CM) | ✅ |
| Trading strategie analýza | ✅ |
| Uložení modelů | ✅ |

### 📁 Vytvořené soubory:

| Soubor | Popis |
|--------|-------|
| `models/price_classifier_global.pkl` | Globální RF klasifikátor |
| `models/{Sector}_price_classifier.pkl` | Sektorové klasifikátory |
| `models/price_classifier_metadata.json` | Metadata modelů |

### ➡️ Další notebook:

**Notebook 05: Hyperparameter Tuning**
- Grid Search s TimeSeriesSplit
- Optimalizace RF parametrů

In [ ]:
# Finální shrnutí
print("="*70)
print("📊 NOTEBOOK 04 - SHRNUTÍ")
print("="*70)

print(f"\n🎯 Klasifikace:")
print(f"   Třídy: {CLASS_NAMES}")
print(f"   Práh: ±{THRESHOLD*100:.0f}%")

print(f"\n📊 Globální Model:")
print(f"   Accuracy: {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f"   F1-Score: {f1:.4f}")

print(f"\n🏭 Sektorové Modely:")
for sector, res in sector_results.items():
    print(f"   {sector}: Acc={res['accuracy']:.3f}, F1={res['f1']:.3f}")

print(f"\n📈 Trading Strategie:")
print(f"   Win Rate: {strategy['win_rate']*100:.1f}%")
print(f"   Outperformance: {strategy['outperformance']*100:+.1f}%")

print(f"\n✅ Modely připraveny pro produkci!")